<a href="https://colab.research.google.com/github/yk-Jeong/nextlab_collabo/blob/main/CP2_%EA%B8%B0%EC%97%85%ED%98%91%EC%97%85_%EB%84%A5%EC%8A%A4%ED%8A%B8%EB%9E%A9(1)_%EB%89%B4%EC%8A%A4%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 뉴스 카테고리 분류**
2. 한글 띄어쓰기 검사
3. 단락 연결 여부 판별

#뉴스 카테고리 분류

9가지 카테고리(culture, economy, entertainment, health, ITscience, life, politic, social, sport)의 다중분류문제 

###라이브러리 설치 및 로드

In [1]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 47.3 MB 6.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595741 sha256=4600d661a88f8149cd1df4766aa8504971902cbab847d63311433114ee867615
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 35.0 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 29.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-qvgefpc5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-qvgefpc5
     |████████████████████████████████| 132 kB 5.4 MB/s 
     |████████████████████████████████| 4.5 MB 34.7 MB/s 
     |████████████████████████████████| 8.6 MB 31.5 MB/s 
     |████████████████████████████████| 79 kB 4.1 MB/s 
     |████████████████████████████████| 127 kB 36.1 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=f608b000a00fa9ce8ff616629e28b537c990a0ac644cf43003be5c739f64f48a
  Stored in directory: /tmp/pip-ephem-wheel-cache-vco96zv2/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [3]:
import torch 
from torch import nn

import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
import numpy as np
import pandas as pd
import os

from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

###데이터 로드 

In [5]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip -uq "/content/drive/My Drive/news_class9x13000.zip" -d "/content/dataset"
!ls 

dataset  drive	sample_data


In [7]:
#아래 라인이 실행되지 않을 경우 숨겨진 파일인 .ipynb_checkpoints를 삭제

!rmdir /content/dataset/2/.ipynb_checkpoints
#참고 https://wooono.tistory.com/252

rmdir: failed to remove '/content/dataset/2/.ipynb_checkpoints': No such file or directory


데이터 확인 및 분리 

In [8]:
#폴더명 변경: dataset 폴더에 있는 하위 카테고리명을 0에서 8까지의 숫자로 변경
#d이미 변경하였다면 미실행

os.listdir('/content/dataset')
os.rename('/content/dataset/culture', '/content/dataset/0')
os.rename('/content/dataset/economy', '/content/dataset/1')
os.rename('/content/dataset/entertainment', '/content/dataset/2')
os.rename('/content/dataset/health', '/content/dataset/3')
os.rename('/content/dataset/ITscience', '/content/dataset/4')
os.rename('/content/dataset/life', '/content/dataset/5')
os.rename('/content/dataset/politic', '/content/dataset/6')
os.rename('/content/dataset/social', '/content/dataset/7')
os.rename('/content/dataset/sport', '/content/dataset/8')

FileNotFoundError: ignored

In [9]:
#기사 내용과 카테고리를 하나의 데이터로 이루어진 리스트로 만듦

dataset_train = []
dataset_test = []

root = "/content/dataset/"
list = os.listdir(root)
for cat in list:
    files = os.listdir(root + cat)
    for i,f in enumerate(files):
        fname = root + cat + "/" + f
        file = open(fname, "r", encoding="utf-8")
        strings = file.read()
        if i>=2600: #카테고리당 13,000개의 데이터가 있으며 훈련 세트의 비중을 20%로 설정하였음
            dataset_train.append([strings, cat])
        else:
            dataset_test.append([strings, cat])
        file.close()

print(len(dataset_train), len(dataset_test))

93600 23400


In [10]:
dataset_train[0], dataset_test[0]

(['컴백 강호동, 너무 식상하지 않을까?\n지난해 세금 탈세 혐의에 연루돼 연예계 잠정 은퇴를 선언한 강호동이 1년 2개월 만에 MC로 복귀한다.\n강호동이 지난 9월 SM엔터테인먼트의 계열사 SM C&C와 정식 계약을 체결하고 컴백을 타진하고 나선 것.\n복귀 선언 한 달여 만에 강호동은 SBS ‘스타킹’, MBC ‘황금어장-무릎팍 도사’, KBS의 새 예능프로그램으로의 컴백이 확정됐다.\n오는 29일 진행될 ‘스타킹’ 첫 녹화를 시작으로 본격적으로 MC 마이크를 잡게 됐다.\n방송가 내에서 강호동의 복귀 시점에 대해 ‘너무 시급하다’는 반론도 만만치 않다.\n또 강호동이 복귀함에 따라 다시금 유재석과 투톱 체제를 이어나가는 것에 대한 우려도 있다.\n그러나 MC계는 별다른 이변이 없는 한 유재석-강호동의 양강 구도가 재현될 것으로 보고있다.\n◈ 강호동과 유재석은 독보적 존재?\n방송사가 다른 여러 MC들보다 강호동과 유재석을 선호하는 이유는 뭘까?\n바로 연예계 누구에게도 없는 MC로서 적합하고, 탁월한 능력을 지녔기 때문이다.\n잠정 은퇴 선언 전까지 강호동이 진행했던 프로그램을 연출한 한 PD는 “강호동은 여러 MC들 중에 온 국민을 포괄할 수 있는 독보적인 MC라며 ”다른 MC로 대체 불가능한 본인만의 스타일이 있다”고 말했다.\n강호동과 유재석은 약 10년간 방송 3사 연말 연예대상 때마다 두 사람은 번갈아 가면서 대상을 휩쓸 정도였다.\n이들의 진행 능력과 예능감은 물론 ‘최고’라는 평가가 많지만, 일각에서는 식상하다는 지적도 많다.\n실제로 1년 전만 해도 강호동과 유재석은 각각 4개의 프로그램을 진행하면서 TV에 자주 모습을 비쳤다.\n이와 같은 현상을 두고 강호동 컴백작의 기획을 맡게 된 한 CP는 “강호동-유재석은 10여년 가까이 예능계를 이끄는 투톱으로 그만큼 노출이 많았기 때문에 식상함을 유발할 수 있는 것이 사실”이라고 밝혔다.\n또 강호동-유재석이 출연했던 프로그램을 제작한 PD는 “두 사람은 워낙 독보적이고 각 방송사에서 모두 

######tokenizer 선언

In [11]:
#device setting 및 사전학습된 bert model load
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


######데이터셋 클래스 선언

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

######모델 생성

In [16]:
#params

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()

        self.bert = bert
        self.dr_rate = dr_rate                 
        self.classifier = nn.Linear(hidden_size , num_classes)

        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
#accuracy 계산 
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1463 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.7866648154477102


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.8785177595628416


  0%|          | 0/1463 [00:00<?, ?it/s]

epoch 3 train acc 0.9449226760082023


  0%|          | 0/366 [00:00<?, ?it/s]

epoch 3 test acc 0.877834699453552


testModel: 모델 테스트를 위한 함수

In [22]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq): 
  categories = ['culture', 'economy', 'entertainment', 'health', 'ITscience', 'life', 'politic', 'social', 'sport']
  tmp = [seq]
  transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
  tokenized = transform(tmp)
  
  model.eval()
  result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
  idx = result.argmax().cpu().item()
  print("category:", categories[idx])
  print("accuracy:", "{:.2f}%".format(softmax(result,idx)))

포털 사이트에서 기사문 제목을 수집하여 모델을 테스트 

In [23]:
testModel(model, "겨우 13건 신청…'양육비 미지급자 공개' 유명무실") #실제 

#https://www.sedaily.com/NewsView/2623NRMOF3

category: economy
accuracy: 58.05%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  from ipykernel import kernelapp as app


In [24]:
testModel(model, "여가부 없애려는 한국 보수, 자기개혁 실패 보여주는 결정적 사례")

#https://www.hani.co.kr/arti/society/women/1036459.html

category: politic
accuracy: 82.30%


In [25]:
testModel(model, "러 '국가 존립에 위협 있을 때만 핵무기 사용할 것'")

#https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=104&oid=055&aid=0000963131

category: social
accuracy: 99.27%


In [26]:
testModel(model, "축구장 면적 14배…LG CNS, '죽전 데이터센터' 사업 수주")

#https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=018&aid=0005176974

category: ITscience
accuracy: 93.77%


In [27]:
testModel(model, "당장 떠날 수 있는 무격리 여행지는 어디?")

#https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=103&oid=421&aid=0005996309

category: life
accuracy: 98.08%
